In [1]:
import sys
print(sys.path)

from src.training import SegmentationTrainer, TrainingConfig
from src.models.unet_mobilenet import build_unet_mobilenet

['/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages']


In [ ]:
config = TrainingConfig(
      data_root="datasets/NYUDepthv2",
      use_rgb=True,
      use_depth=False,
      lr=1e-3,
      epochs=20,
      batch_size=32,
      early_stopping_enabled=True,
      early_stopping_patience=5,
      device="auto",
      save_best_path="checkpoints/unet_mobilenet/best3.pth",
      save_latest_path="checkpoints/unet_mobilenet/latest2.pth",
      num_classes=40,
)

# Create model and train
model = build_unet_mobilenet(num_classes=config.num_classes)

trainer = SegmentationTrainer(config)

Loading ImageNet pretrained MobileNetV3-Small weights...
✓ Successfully loaded ImageNet pretrained MobileNetV3-Small weights
UNet-MobileNetV3-SMALL initialized:
  Total parameters: 2,361,960
  Trainable parameters: 2,361,960
  Encoder channels: [16, 16, 24, 48, 96]
Using device: mps
EarlyStopping: monitoring 'val_miou' with patience=5 (mode=max)


In [3]:
model.freeze_backbone()
trained_model = trainer.train(model)

MobileNetV3-small encoder frozen: parameters will not be updated during training
Training configuration:
  Use RGB: True
  Use Depth: False
  Input channels: 3
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 2,361,960
Trainable parameters: 1,434,952
Starting training for 20 epochs (from epoch 1)...


/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoch 1 | Iter 00001] loss=3.7191
[Epoch 1 | Iter 00002] loss=3.7004
[Epoch 1 | Iter 00003] loss=3.6970
[Epoch 1 | Iter 00004] loss=3.6887
[Epoch 1 | Iter 00005] loss=3.6804
[Epoch 1 | Iter 00006] loss=3.6748
[Epoch 1 | Iter 00007] loss=3.6645
[Epoch 1 | Iter 00008] loss=3.6565
[Epoch 1 | Iter 00009] loss=3.6479
[Epoch 1 | Iter 00010] loss=3.6387
[Epoch 1 | Iter 00011] loss=3.6281
[Epoch 1 | Iter 00012] loss=3.6180
[Epoch 1 | Iter 00013] loss=3.6075
[Epoch 1 | Iter 00014] loss=3.5979
[Epoch 1 | Iter 00015] loss=3.5875
[Epoch 1 | Iter 00016] loss=3.5771
[Epoch 1 | Iter 00017] loss=3.5666
[Epoch 1 | Iter 00018] loss=3.5548
[Epoch 1 | Iter 00019] loss=3.5445
[Epoch 1 | Iter 00020] loss=3.5347
[Epoch 1 | Iter 00021] loss=3.5250
[Epoch 1 | Iter 00022] loss=3.5126
[Epoch 1 | Iter 00023] loss=3.5003
[Epoch 1 | Iter 00024] loss=3.4887
[Epoch 1 | Iter 00025] loss=3.4775
[Epoch 1] Train: loss=3.4775 (28.7s, 27.7 samples/s) | Val: mIoU=0.0252 PixelAcc=0.3587 (19.6s, 5.1 samples/s)
✓ Saved best c

In [4]:
model.unfreeze_backbone()
config_stage_2 = config
config_stage_2.epochs = 30
config_stage_2.lr = 1e-4 # lower learning rate for fine-tuning
trainer = SegmentationTrainer(config_stage_2)
trained_model = trainer.train(model)

MobileNetV3-small encoder unfrozen: parameters will be updated during training
Using device: mps
Training configuration:
  Use RGB: True
  Use Depth: False
  Input channels: 3
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 2,361,960
Trainable parameters: 2,361,960
Resuming training from: checkpoints/unet_mobilenet/latest2.pth
✓ Resumed from epoch 20
✓ Training history loaded with 20 epochs
✓ Best mIoU so far: 0.0894
Starting training for 30 epochs (from epoch 21)...
[Epoch 21 | Iter 00001] loss=1.3446
[Epoch 21 | Iter 00002] loss=1.3771
[Epoch 21 | Iter 00003] loss=1.4331
[Epoch 21 | Iter 00004] loss=1.4333
[Epoch 21 | Iter 00005] loss=1.4436
[Epoch 21 | Iter 00006] loss=1.4600
[Epoch 21 | Iter 00007] loss=1.4629
[Epoch 21 | Iter 00008] loss=1.4803
[Epoch 21 | Iter 00009] loss=1.4754
[Epoch 21 | Iter 00010] loss=1.4899
[Epoch 21 | Iter 00011] loss=1.4888
[Epoch 21 | Iter 00012] loss=1.4913
[Epoch 21 | Iter 00013] loss=1.4926
[Epoch

In [3]:
# reload SegmentationTrainer with new config
model.unfreeze_backbone()
config_stage_2 = config
config_stage_2.epochs = 40
config_stage_2.lr = 1e-4 # lower learning rate for fine-tuning
trainer = SegmentationTrainer(config_stage_2)
trained_model = trainer.train(model)

MobileNetV3-small encoder unfrozen: parameters will be updated during training
Using device: mps
Training configuration:
  Use RGB: True
  Use Depth: False
  Input channels: 3
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 2,361,960
Trainable parameters: 2,361,960
Resuming training from: checkpoints/unet_mobilenet/latest2.pth
✓ Resumed from epoch 30
✓ Training history loaded with 30 epochs
✓ Best mIoU so far: 0.1367
🔧 Applied config overrides:
  learning_rate: 0.001000 → 0.000100
Starting training for 40 epochs (from epoch 31)...


/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoch 31 | Iter 00001] loss=0.8898
[Epoch 31 | Iter 00002] loss=0.9343
[Epoch 31 | Iter 00003] loss=0.9540
[Epoch 31 | Iter 00004] loss=0.9425
[Epoch 31 | Iter 00005] loss=0.9420
[Epoch 31 | Iter 00006] loss=0.9426
[Epoch 31 | Iter 00007] loss=0.9390
[Epoch 31 | Iter 00008] loss=0.9503
[Epoch 31 | Iter 00009] loss=0.9527
[Epoch 31 | Iter 00010] loss=0.9474
[Epoch 31 | Iter 00011] loss=0.9365
[Epoch 31 | Iter 00012] loss=0.9348
[Epoch 31 | Iter 00013] loss=0.9351
[Epoch 31 | Iter 00014] loss=0.9344
[Epoch 31 | Iter 00015] loss=0.9318
[Epoch 31 | Iter 00016] loss=0.9307
[Epoch 31 | Iter 00017] loss=0.9302
[Epoch 31 | Iter 00018] loss=0.9251
[Epoch 31 | Iter 00019] loss=0.9281
[Epoch 31 | Iter 00020] loss=0.9283
[Epoch 31 | Iter 00021] loss=0.9285
[Epoch 31 | Iter 00022] loss=0.9248
[Epoch 31 | Iter 00023] loss=0.9185
[Epoch 31 | Iter 00024] loss=0.9170
[Epoch 31 | Iter 00025] loss=0.9135
[Epoch 31] Train: loss=0.8756 mIoU=0.3556 pixacc=0.7464 (35.9s+24.4s, 22.2 smp/s) | Val: loss=1.7044

In [3]:
# now do so with early stopping
model.unfreeze_backbone()
config_stage_3 = config
config_stage_3.epochs = 50
config_stage_3.lr = 1e-4 
trainer = SegmentationTrainer(config_stage_3)
trained_model = trainer.train(model)

MobileNetV3-small encoder unfrozen: parameters will be updated during training
Using device: mps
EarlyStopping: monitoring 'val_miou' with patience=5 (mode=max)
Training configuration:
  Use RGB: True
  Use Depth: False
  Input channels: 3
  Label mode: nyu40
  Num classes: 40
Training samples: 795
Validation samples: 100
Total parameters: 2,361,960
Trainable parameters: 2,361,960
Resuming training from: checkpoints/unet_mobilenet/latest2.pth
✓ Resumed from epoch 40
✓ Training history loaded with 40 epochs
✓ Best mIoU so far: 0.1732
✓ No optimizer parameter changes needed
Starting training for 50 epochs (from epoch 41)...


/Users/reubendrummond/Desktop/Uni/2025/Sem2/EGH444/EGH444-Group-10/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[Epoch 41 | Iter 00001] loss=0.7218
[Epoch 41 | Iter 00002] loss=0.7464
[Epoch 41 | Iter 00003] loss=0.7663
[Epoch 41 | Iter 00004] loss=0.7557
[Epoch 41 | Iter 00005] loss=0.7623
[Epoch 41 | Iter 00006] loss=0.7714
[Epoch 41 | Iter 00007] loss=0.7703
[Epoch 41 | Iter 00008] loss=0.7830
[Epoch 41 | Iter 00009] loss=0.7879
[Epoch 41 | Iter 00010] loss=0.7843
[Epoch 41 | Iter 00011] loss=0.7767
[Epoch 41 | Iter 00012] loss=0.7764
[Epoch 41 | Iter 00013] loss=0.7783
[Epoch 41 | Iter 00014] loss=0.7791
[Epoch 41 | Iter 00015] loss=0.7783
[Epoch 41 | Iter 00016] loss=0.7788
[Epoch 41 | Iter 00017] loss=0.7794
[Epoch 41 | Iter 00018] loss=0.7752
[Epoch 41 | Iter 00019] loss=0.7787
[Epoch 41 | Iter 00020] loss=0.7805
[Epoch 41 | Iter 00021] loss=0.7823
[Epoch 41 | Iter 00022] loss=0.7803
[Epoch 41 | Iter 00023] loss=0.7758
[Epoch 41 | Iter 00024] loss=0.7752
[Epoch 41 | Iter 00025] loss=0.7733
[Epoch 41] Train: loss=0.7339 mIoU=0.4419 pixacc=0.7923 (36.6s+25.1s, 21.7 smp/s) | Val: loss=1.6903